### STEP: 1

Import necessary libraries for **Predicting Future Stock Prices**

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
import requests

import warnings
warnings.filterwarnings('ignore')

# Libraries to Extract Stock Price Historical Data 
# from Yahoo! Finance using Tickers from Wikipedia
import yfinance as yf
from pandas.tseries.offsets import DateOffset
from datetime import datetime, timedelta
from pandas.tseries.holiday import USFederalHolidayCalendar

# Library for Technical Analysis
import pandas_ta as ta

# Libraries for ML Analysis using Linear Models
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import MultiTaskElasticNet
from sklearn.linear_model import LassoLars
from sklearn.linear_model import BayesianRidge

scaler = StandardScaler()
today = datetime.today().strftime('%Y-%m-%d')

### STEP: 2

Defining various functions used in the code for 
* extracting data, 
* adding technical indicators as features,
* predicting **Close** price and **VWAP** volume-weighted average price,
* using linear models for predicting with best model output.

Best model is selected based on minimum MAPE & RMSE

In [ ]:
# Extracting Stock Price Historical Data from Yahoo! Finance using Tickers from Wikipedia
        
def extract_stock_data(stock_name, start_date):
    row_index = ticker_df.query(f'Security == "{stock_name}"').index[0]
    stock_ticker = ticker_list[row_index]
    OHLCV_df = yf.download(stock_ticker,start_date,today)
    OHLCV_df.index = pd.to_datetime(OHLCV_df.index)
    #print(f"{stock_ticker}:{stock_name}")
    #display(OHLCV_df.head(3))
    #display(OHLCV_df.tail(3))
    return(OHLCV_df)

# Feature Engineering _ additionals features for ML Analysis

def technical_indicators():
    OHLCV_df = extract_stock_data(stock_name, start_date)
    # VWAP modified to use Annual cummulative sum
    OHLCV_df['typical_price'] = (OHLCV_df['High']+OHLCV_df['Low']+OHLCV_df['Close'])/3
    OHLCV_df['vw_typical_price'] = OHLCV_df['typical_price']*OHLCV_df['Volume']
    OHLCV_df['vwap_annual'] = OHLCV_df.groupby(OHLCV_df.index.year)['vw_typical_price'].cumsum() / OHLCV_df.groupby(OHLCV_df.index.year)['Volume'].cumsum()
    # Daily Returns
    OHLCV_df['daily_returns'] = OHLCV_df['Close'].pct_change()
    # 21Day Volatility
    OHLCV_df['volatility_21'] = OHLCV_df['Close'].rolling(window=21).std()
    # Simple Moving Avg 21Days & 50Days
    OHLCV_df['sma_21'] = OHLCV_df.ta.sma(length=21)
    OHLCV_df['sma_50'] = OHLCV_df.ta.sma(length=50)
    # Exponential Moving Avg 21Days & 50Days
    OHLCV_df['ema_21'] = OHLCV_df.ta.ema(length=21)
    OHLCV_df['ema_50'] = OHLCV_df.ta.ema(length=50)
    # RSI Momemtum Indicator 14Days
    rsi_df = OHLCV_df.ta.rsi()
    # MACD Momentum Indicator
    macd_df = OHLCV_df.ta.macd()
    # Bolinger Bands %B Indicator
    bbands_df = OHLCV_df.ta.bbands()
    OHLCV_append_df = pd.concat([OHLCV_df,rsi_df,macd_df,bbands_df],axis=1)    
    return(OHLCV_append_df)


In [ ]:
# Predicting Stock Close & Volume-Weighted Average Price(VWAP) using Linear Models

def predict_close_vwap():
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    predictors_df = predictors_df.drop(columns = ['Open','High','Low','Adj Close','Volume',
                   'typical_price','vw_typical_price']).dropna()
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    # Training Data of 11months and Testing Data of 22months alternatively 
    # with latest 33months of testing data
    train_range_months = 11
    test_range_months = 22
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_test_scaled = predictors_scaled.transform(predictors_test)

    model = BayesianRidge()
    target_predictions_close_df = target_test_close.copy()
    model.fit(predictors_train_scaled,target_train_close)
    model_prediction_close = model.predict(predictors_test_scaled)
    target_predictions_close_df['Bayesian_Close'] = model_prediction_close

    target_predictions_vwap_df = target_test_vwap.copy()
    model.fit(predictors_train_scaled,target_train_vwap)
    model_prediction_vwap = model.predict(predictors_test_scaled)
    target_predictions_vwap_df['Bayesian_vwap_annual'] = model_prediction_vwap

    target_predictions_df = pd.concat([target_predictions_close_df,target_predictions_vwap_df],axis=1)

    return(target_predictions_df)



In [ ]:
# Calculating Root_Mean_Square_Error(RMSE) 
# and Mean_Absolute_Percentage_Error(MAPE)

def linear_model_predictions():
    target_predictions_df = predict_close_vwap()
    dict_close = {}
    dict_vwap = {}
    rmse_close = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,'Bayesian_Close'])**2))
    mape_close = np.mean(np.abs(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,'Bayesian_Close'])/target_predictions_df.loc[:today,'Close']))*100
    dict_close['Bayesian'] = [rmse_close,mape_close]
    rmse_vwap = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,'Bayesian_vwap_annual'])**2))
    mape_vwap = np.mean(np.abs(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,'Bayesian_vwap_annual'])/target_predictions_df.loc[:today,'vwap_annual']))*100
    dict_vwap['Bayesian_vwap_annual'] = [rmse_vwap,mape_vwap]
    error_measures_close = pd.DataFrame.from_dict(dict_close, orient = 'index', columns = ['RMSE_close','MAPE_close'])
    error_measures_vwap = pd.DataFrame.from_dict(dict_vwap, orient = 'index', columns = ['RMSE_vwap','MAPE_vwap'])
    
    predictions_output = target_predictions_df[['Close','vwap_annual','Bayesian_Close','Bayesian_vwap_annual']]
    
    nxt_day = predictions_output.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    predictions_output.loc[nxt_day] = np.nan
    predictions_output = predictions_output.shift()
    predictions_output['Close']=predictions_output['Close'].fillna(predictions_output['Bayesian_Close'])
    predictions_output['vwap_annual']=predictions_output['vwap_annual'].fillna(predictions_output['Bayesian_vwap_annual'])

    nxt_day_predictions = predictions_output[['Close','vwap_annual']]
    model_evaluation = error_measures_close
    #display(predictions_output[['Close',best_estimate_close]].loc['2024':today].hvplot(frame_width=500,frame_height=300))
    
    return (model_evaluation,nxt_day_predictions)


In [ ]:
# Feature Engineering _ for future prediction using stock_predictions as input
# OBV & ADX dropped to limit predicted features

def feature_engineering(df):
    # Daily Returns
    df['daily_returns'] = df['Close'].pct_change()
    # 21Day Volatility
    df['volatility_21'] = df['Close'].rolling(window=21).std()
    # Simple Moving Avg 21Days & 50Days
    df['sma_21'] = df['Close'].rolling(window=21).mean()
    df['sma_50'] = df['Close'].rolling(window=50).mean()
    # Exponential Moving Avg 21Days & 50Days
    df['ema_21'] = df['Close'].ewm(span=21, adjust=False).mean()
    df['ema_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    # RSI Momemtum Indicator 14Days
    rsi_df = df.ta.rsi()
    # MACD Indicator
    macd_df = df.ta.macd()
    # Bolinger Bands %B Indicator
    bbands_df = df.ta.bbands()
    # Dropping trivial Columns
    append_df = pd.concat([df,rsi_df,macd_df,bbands_df],axis=1).dropna()
    # trim_df = df[['Close','vwap_annual']]
    return(append_df)


In [ ]:
# Creating instance of best fit model _ BayesianRidge()
# By fitting the historical data

def best_fit_model_future_prediction(df):
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    predictors_df = predictors_df.drop(columns = ['Open','High','Low','Adj Close','Volume',
                   'typical_price','vw_typical_price']).dropna()
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    predictors_future_df = feature_engineering(df)
    target_future_close_df = predictors_future_df[['Close']].shift(-1)
    target_future_vwap_df = predictors_future_df[['vwap_annual']].shift(-1)

    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 11
    test_range_months = 22
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_future_scaled = predictors_scaled.transform(predictors_future_df)

    best_model = BayesianRidge()
    target_future_close = target_future_close_df.copy()
    model_instance_close = best_model
    model_instance_close.fit(predictors_train_scaled,target_train_close)
    model_prediction_close = model_instance_close.predict(predictors_future_scaled)
    target_future_close[f'{best_model}_Close'] = model_prediction_close[-1]

    target_future_vwap = target_future_vwap_df.copy()
    model_instance_vwap = best_model
    model_instance_vwap.fit(predictors_train_scaled,target_train_vwap)
    model_prediction_vwap = model_instance_vwap.predict(predictors_future_scaled)
    target_future_vwap[f'{best_model}_vwap_annual'] = model_prediction_vwap[-1]

    target_future_df = pd.concat([target_future_close,target_future_vwap],axis=1)

    #display(target_future_df.tail(1))

    nxt_day = target_future_df.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    target_future_df.loc[nxt_day] = np.nan
    target_future_df = target_future_df.shift()
    target_future_df['Close']=target_future_df['Close'].fillna(target_future_df[f'{best_model}_Close'])
    target_future_df['vwap_annual']=target_future_df['vwap_annual'].fillna(target_future_df[f'{best_model}_vwap_annual'])

    future_predictions = target_future_df[['Close','vwap_annual']]
    return (future_predictions)


In [ ]:
# Scrapping for List of stocks in S&P500 for Linear Model ML Prediction Analysis
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
  
wiki_data = pd.read_html(url)

ticker_df = wiki_data[0]
#BRK.B & BF.B DROPPED AS DATA NOT REPORTED THROUGH YFINANCE
#'Constellation Energy','Kenvue','Veralto' Dropped due to less historical Data
ticker_df = ticker_df.drop(index=[62,78,126,212,273,468]).reset_index()
ticker_df['Date added']=pd.to_datetime(ticker_df['Date added'])
ticker_df = ticker_df.sort_values(by=['Date added']).reset_index()
ticker_df = ticker_df.drop(columns=['index','level_0'])
display(ticker_df.head(2))
display(ticker_df.tail(2))

# Pick Stock from the list for analysis
ticker_list= ticker_df['Symbol'].to_list()
stock_list = ticker_df['Security'].to_list()




In [ ]:
# display predictions
master_df=pd.DataFrame()
model_df=pd.DataFrame(index=stock_list, columns=['RMSE','MAPE'])

for stock in stock_list:
    stock_name = stock
    start_date ='2014-01-01'
    
    model,stock_predictions = linear_model_predictions()
    model_df.loc[stock,'RMSE']= round(model.iloc[0,0],2)
    model_df.loc[stock,'MAPE']=round(model.iloc[0,1],2)
    
    future_stock_price = best_fit_model_future_prediction(stock_predictions)
    # Predicting Stocks on Future Dates _ Next 14 Days
    for i in list(range(1,14)):
        future_stock_price = best_fit_model_future_prediction(future_stock_price)
        
    master_df[f'{stock}'] = future_stock_price['Close']




In [ ]:
predicted_price = master_df.copy()
model_eval = model_df.copy()

In [ ]:
predicted_price

In [ ]:
st_list = ['Lululemon Athletica','Hubbell Incorporated','Builders FirstSource',
        'Jabil','Uber','Deckers Brands','Supermicro']
# display predictions
master_df=pd.DataFrame()
model_df=pd.DataFrame(index=stock_list, columns=['RMSE','MAPE'])

for stock in st_list:
    stock_name = stock
    start_date ='2014-01-01'
    
    model,stock_predictions = linear_model_predictions()
    model_df.loc[stock,'RMSE']= round(model.iloc[0,0],2)
    model_df.loc[stock,'MAPE']=round(model.iloc[0,1],2)
    
    future_stock_price = best_fit_model_future_prediction(stock_predictions)
    # Predicting Stocks on Future Dates _ Next 14 Days
    for i in list(range(1,14)):
        future_stock_price = best_fit_model_future_prediction(future_stock_price)
        
    master_df[f'{stock}'] = future_stock_price['Close']

In [ ]:
final_df=pd.concat([predicted_price,master_df],axis=1)
final_df
model_final = pd.concat([model_eval,model_df],axis=0)
model_final

In [ ]:
final_df.to_csv('predicted_price.csv',index=True)

In [ ]:
model_final.to_csv('model_eval.csv',index=True)

## Predicting Using Regressor Models

In [ ]:
# Single Target Regressor Models
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

svr_rbf = SVR(kernel='rbf',C=100,gamma='auto')
# svr_poly = SVR(kernel='poly',C=100,degree=2, gamma='auto')

knn_uniform = KNeighborsRegressor(n_neighbors=2,weights='uniform')
knn_distance = KNeighborsRegressor(n_neighbors=5,weights='distance')

dtree = DecisionTreeRegressor(max_depth=10)

ens = MultiOutputRegressor(HistGradientBoostingRegressor(random_state=0))

# mlp = MLPRegressor(random_state=1, max_iter=100)

In [ ]:
# Predicting Stock Close & Volume-Weighted Average Price(VWAP) using Linear Models

def predict_regressor_close_vwap():
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 27
    test_range_months = 6
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_test_scaled = predictors_scaled.transform(predictors_test)

    list_of_regressors = [ens]
    target_predictions_close_df = target_test_close.copy()
    for model in list_of_regressors:
        model_instance_close = model
        model_instance_close.fit(predictors_train_scaled,target_train_close)
        model_prediction_close = model_instance_close.predict(predictors_test_scaled)
        target_predictions_close_df['ens_Close'] = model_prediction_close

    target_predictions_vwap_df = target_test_vwap.copy()
    for model in list_of_regressors:
        model_instance_vwap = model
        model_instance_vwap.fit(predictors_train_scaled,target_train_vwap)
        model_prediction_vwap = model_instance_vwap.predict(predictors_test_scaled)
        target_predictions_vwap_df['ens_vwap_annual'] = model_prediction_vwap

    target_predictions_df = pd.concat([target_predictions_close_df,target_predictions_vwap_df],axis=1)

    return(target_predictions_df)


In [ ]:
# Calculating Root_Mean_Square_Error(RMSE) 
# and Mean_Absolute_Percentage_Error(MAPE)

def regressor_model_predictions():
    target_predictions_df = predict_regressor_close_vwap()
    dict_close = {}
    dict_vwap = {}
    rmse_close = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,'ens_Close'])**2))
    mape_close = np.mean(np.abs(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,'ens_Close'])/target_predictions_df.loc[:today,'Close']))*100
    dict_close['ens'] = [rmse_close,mape_close]
  
    rmse_vwap = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,'ens_vwap_annual'])**2))
    mape_vwap = np.mean(np.abs(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,'ens_vwap_annual'])/target_predictions_df.loc[:today,'vwap_annual']))*100
    dict_vwap['ens'] = [rmse_vwap,mape_vwap]
    error_measures_close = pd.DataFrame.from_dict(dict_close, orient = 'index', columns = ['RMSE_close','MAPE_close'])
    error_measures_vwap = pd.DataFrame.from_dict(dict_vwap, orient = 'index', columns = ['RMSE_vwap','MAPE_vwap'])

    predictions_output = target_predictions_df[['Close','vwap_annual','ens_Close','ens_vwap_annual']]
    
    nxt_day = predictions_output.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    predictions_output.loc[nxt_day] = np.nan
    predictions_output = predictions_output.shift()
    predictions_output['Close']=predictions_output['Close'].fillna(predictions_output['ens_Close'])
    predictions_output['vwap_annual']=predictions_output['vwap_annual'].fillna(predictions_output['ens_vwap_annual'])

    predictions = predictions_output[['Close','vwap_annual']].dropna()
    best_model = error_measures_close

    return (best_model,predictions)



In [ ]:
# Creating instance of best fit regressor model
# By fitting the historical data

def best_fit_regr_future_prediction(df):
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    predictors_df = predictors_df.drop(columns = ['Open','High','Low','Adj Close','Volume',
                   'typical_price','vw_typical_price']).dropna()
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    predictors_future_df = feature_engineering(df)
    target_future_close_df = predictors_future_df[['Close']].shift(-1)
    target_future_vwap_df = predictors_future_df[['vwap_annual']].shift(-1)

    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 27
    test_range_months = 6
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_future_scaled = predictors_scaled.transform(predictors_future_df)

    best_model_close = ens
    target_future_close = target_future_close_df.copy()
    model_instance_close = best_model_close
    model_instance_close.fit(predictors_train_scaled,target_train_close)
    model_prediction_close = model_instance_close.predict(predictors_future_scaled)
    target_future_close[f'{best_model_close}_Close'] = model_prediction_close

    best_model_vwap = ens
    target_future_vwap = target_future_vwap_df.copy()
    model_instance_vwap = best_model_vwap
    model_instance_vwap.fit(predictors_train_scaled,target_train_vwap)
    model_prediction_vwap = model_instance_vwap.predict(predictors_future_scaled)
    target_future_vwap[f'{best_model_vwap}_vwap_annual'] = model_prediction_vwap

    target_future_df = pd.concat([target_future_close,target_future_vwap],axis=1)

    nxt_day = target_future_df.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    target_future_df.loc[nxt_day] = np.nan
    target_future_df = target_future_df.shift()
    target_future_df['Close']=target_future_df['Close'].fillna(target_future_df[f'{best_model_close}_Close'])
    target_future_df['vwap_annual']=target_future_df['vwap_annual'].fillna(target_future_df[f'{best_model_vwap}_vwap_annual'])

    predictions = target_future_df[['Close','vwap_annual']]
    
    return (predictions)


In [65]:
# display predictions
master_regr_df=pd.DataFrame()
model_regr_df=pd.DataFrame(index=stock_list, columns=['RMSE','MAPE'])

for stock in stock_list:
    stock_name = stock
    start_date ='2014-01-01'
    
    model,stock_predictions = regressor_model_predictions()
    model_regr_df.loc[stock,'RMSE']= round(model.iloc[0,0],2)
    model_regr_df.loc[stock,'MAPE']=round(model.iloc[0,1],2)
    
    future_stock_price = best_fit_regr_future_prediction(stock_predictions)
    # Predicting Stocks on Future Dates _ Next 14 Days
    for i in list(range(1,14)):
        future_stock_price = best_fit_regr_future_prediction(future_stock_price)
        
    master_regr_df[f'{stock}'] = future_stock_price['Close']


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

ValueError: Found array with 0 sample(s) (shape=(0, 17)) while a minimum of 1 is required by StandardScaler.

In [67]:
predicted_regr_price = master_regr_df.copy()
model_regr_eval = model_regr_df.copy()

In [72]:
for stock in stock_list[-100:]:
    stock_name = stock
    start_date ='2014-01-01'
    
    model,stock_predictions = regressor_model_predictions()
    model_regr_df.loc[stock,'RMSE']= round(model.iloc[0,0],2)
    model_regr_df.loc[stock,'MAPE']=round(model.iloc[0,1],2)
    
    future_stock_price = best_fit_regr_future_prediction(stock_predictions)
    # Predicting Stocks on Future Dates _ Next 14 Days
    for i in list(range(1,14)):
        future_stock_price = best_fit_regr_future_prediction(future_stock_price)
        
    master_regr_df[f'{stock}'] = future_stock_price['Close']

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

ValueError: Found array with 0 sample(s) (shape=(0, 17)) while a minimum of 1 is required by StandardScaler.

In [74]:
predicted_regr_price

,3M,General Dynamics,Union Pacific Corporation,Public Service Enterprise Group,Ford Motor Company,ExxonMobil,Exelon,RTX Corporation,Entergy,Archer-Daniels-Midland,...,Martin Marietta Materials,Universal Health Services,United Rentals,Royal Caribbean Group,HCA Healthcare,Skyworks Solutions,Henry Schein,Equinix,American Airlines Group,Realty Income
Date,,,,,,,,,,,,,,,,,,,,,
2023-07-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-20,104.290001,216.080002,216.779999,64.459999,14.030000,103.410004,42.009998,97.160004,103.720001,82.629997,...,458.070007,151.419998,458.739990,101.110001,286.910004,111.480003,79.910004,810.510010,17.440001,62.619999
2023-07-21,104.330002,215.470001,216.779999,65.129997,13.930000,103.889999,42.549999,96.389999,103.989998,82.139999,...,457.890015,148.509995,450.429993,102.379997,285.140015,112.980003,79.919998,807.119995,17.420000,62.750000
2023-07-24,104.269997,217.639999,216.940002,64.559998,13.960000,105.519997,42.830002,97.010002,104.459999,83.110001,...,458.510010,148.320007,450.209991,102.930000,285.179993,112.589996,79.410004,819.099976,16.879999,63.369999
2023-07-25,109.830002,218.000000,215.550003,64.879997,13.580000,105.680000,43.020000,87.099998,104.449997,86.050003,...,458.260010,149.639999,450.220001,101.620003,285.529999,112.879997,80.139999,816.469971,16.650000,63.090000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-12,140.884865,220.335102,234.574948,62.902811,12.590665,95.733250,36.301335,95.391489,106.732914,66.782720,...,417.230096,145.720208,355.142404,126.894733,243.263345,106.567103,75.405283,778.439472,15.724073,56.094162
2024-04-15,141.419175,220.369841,234.812569,62.893637,12.609199,95.405102,36.415653,94.564852,106.700447,66.793878,...,417.294614,145.691066,354.370175,125.508023,240.892432,106.474530,75.597131,790.404619,15.449366,56.394537
2024-04-16,141.415371,220.609513,234.818530,62.634830,12.606007,94.299755,36.427896,94.958795,106.339478,66.985433,...,417.658314,144.585686,348.981330,124.493001,244.918992,106.777136,75.830238,791.088461,15.501612,56.472842


In [76]:
model_regr_eval[341:]

,RMSE,MAPE
Realty Income,1.09,1.32
Qorvo,2.62,2.07
WestRock,NaN,NaN
J.B. Hunt,NaN,NaN
Kraft Heinz,NaN,NaN
...,...,...
Builders FirstSource,NaN,NaN
Jabil,NaN,NaN
Uber,NaN,NaN
Deckers Brands,NaN,NaN


In [ ]:
st_list = ['Lululemon Athletica','Hubbell Incorporated','Builders FirstSource',
        'Jabil','Uber','Deckers Brands','Supermicro']
# display predictions
master_regr_df=pd.DataFrame()
model_regr_df=pd.DataFrame(index=st_list, columns=['RMSE','MAPE'])

for stock in st_list:
    stock_name = stock
    start_date ='2014-01-01'
    
    model,stock_predictions = regressor_model_predictions()
    model_regr_df.loc[stock,'RMSE']= round(model.iloc[0,0],2)
    model_regr_df.loc[stock,'MAPE']=round(model.iloc[0,1],2)
    
    future_stock_price = best_fit_regr_future_prediction(stock_predictions)
    # Predicting Stocks on Future Dates _ Next 14 Days
    for i in list(range(1,14)):
        future_stock_price = best_fit_regr_future_prediction(future_stock_price)
        
    master_regr_df[f'{stock}'] = future_stock_price['Close']

In [ ]:
final_regr_df=pd.concat([predicted_regr_price,master_regr_df],axis=1)
final_regr_df
model_regr_final = pd.concat([model_regr_eval,model_regr_df],axis=0)
model_regr_final

In [ ]:
final_regr_df.to_csv('predicted_price_regr.csv',index=True)
model_regr_final.to_csv('regr_model_eval.csv',index=True)

In [ ]:
#Predicting Day1
future_stock_price_regr = best_fit_regr_future_prediction(stock_predictions_regr)
# Predicting Stocks on Future Dates _ Next 51 Days
for i in list(range(1,14)):
    future_stock_price_regr = best_fit_regr_future_prediction(future_stock_price_regr)

#Display Regressor Model Predictions
display(future_stock_price_regr.loc[today:,:])

future_plot_regr = future_stock_price_regr.loc[today:,'Close'].hvplot(frame_width=300,frame_height=300)

display(future_plot_regr)
display(regressor_plot*future_plot_regr)

In [ ]:
regressor_plot*future_plot*future_plot_regr

In [ ]:
linear_close = future_stock_price[['Close']].rename(columns={'Close':'Close_Linear'})
regr_close = future_stock_price_regr[['Close']].rename(columns={'Close':'Close_Regr'})

combined_df = pd.concat([linear_close,regr_close],axis=1).dropna()

display(combined_df.loc[today:])

combined_df.loc[today:].hvplot()


In [ ]:

for i in list(range(1,8)):
    future_stock_price_regr=best_fit_regr_future_prediction(future_stock_price_regr)
    future_stock_price_regr



In [ ]:
future_stock_price_regr = future_stock_price_regr[['Close','vwap_annual']].dropna()

future_stock_price_regr